In [2]:
!java -version

openjdk version "1.8.0_362-362"
OpenJDK Runtime Environment (build 1.8.0_362-362-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)


# **IRIS Naive Bayes Classification with Pyspark**

In [1]:
# install
# !pip install Pyspark
!pip install pyspark==3.3.1

# Conexión con el cluster de Spark con pyspark

In [2]:
import os
os.environ["AWS_REGION"]
# os.environ["AWS_ACCESS_KEY_ID"]

'us-east-1'

In [3]:
from pyspark.sql import SparkSession

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4 pyspark-shell'
dst_lh_url  = os.environ.get('DST_LH_URL', 'spark://spark-master-0.spark-headless.demml.svc.cluster.local:7077')
dst_lh_appn = os.environ.get('DST_LH_APPN', 'iris_naive_bayes_classification_pyspark_EDU')
import subprocess
my_pod_ip = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip(' \n\t')
my_pod_ip

spark = SparkSession.builder \
    .master(dst_lh_url) \
    .appName(dst_lh_appn) \
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257,software.amazon.awssdk:s3:2.17.257,software.amazon.awssdk:iam:2.17.257,org.apache.hadoop:hadoop-aws:3.2.3') \
    .config('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog') \
    .config('spark.sql.catalog.spark_catalog.catalog-impl','org.apache.iceberg.jdbc.JdbcCatalog') \
    .config('spark.sql.catalog.spark_catalog.uri','jdbc:postgresql://postgresql:5432/iceberg') \
    .config('spark.driver.host', my_pod_ip) \
    .config('spark.sql.catalog.spark_catalog.jdbc.useSSL','false') \
    .config('spark.sql.catalog.spark_catalog.jdbc.user','iceberg') \
    .config('spark.sql.catalog.spark_catalog.jdbc.password','iceberg') \
    .config('spark.sql.catalog.spark_catalog.warehouse','s3a://warehouse/') \
    .config('spark.sql.catalog.spark_catalog.s3.endpoint','http://minio:9000') \
    .config('spark.sql.catalog.spark_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.defaultCatalog','spark_catalog') \
    .config('spark.hadoop.fs.s3a.endpoint','http://minio:9000') \
    .config('spark.hadoop.fs.s3a.access.key','admin') \
    .config('spark.hadoop.fs.s3a.secret.key','t4bl4red0nd4') \
    .config('spark.hadoop.fs.s3a.path.style.access','true') \
    .config('spark.hadoop.fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()

import datetime as dt
ini = dt.datetime.now()

spark

In [4]:
# read sklearn inbuilt data
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
iris = iris.frame
iris = spark.createDataFrame(iris)

/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [5]:
iris.show()

+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|     0|
|              4.9|             3.0|              1.4|             0.2|     0|
|              4.7|             3.2|              1.3|             0.2|     0|
|              4.6|             3.1|              1.5|             0.2|     0|
|              5.0|             3.6|              1.4|             0.2|     0|
|              5.4|             3.9|              1.7|             0.4|     0|
|              4.6|             3.4|              1.4|             0.3|     0|
|              5.0|             3.4|              1.5|             0.2|     0|
|              4.4|             2.9|              1.4|             0.2|     0|
|              4.9|             3.1|              1.

In [6]:
iris.printSchema()

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- target: long (nullable = true)



In [7]:
iris.columns

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)',
 'target']

In [8]:
from pyspark.ml.feature import VectorAssembler 

In [9]:
featureassembler= VectorAssembler(inputCols=['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)'], outputCol='Features')

In [10]:
output = featureassembler.transform(iris)

In [11]:
output.show()

+-----------------+----------------+-----------------+----------------+------+-----------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|         Features|
+-----------------+----------------+-----------------+----------------+------+-----------------+
|              5.1|             3.5|              1.4|             0.2|     0|[5.1,3.5,1.4,0.2]|
|              4.9|             3.0|              1.4|             0.2|     0|[4.9,3.0,1.4,0.2]|
|              4.7|             3.2|              1.3|             0.2|     0|[4.7,3.2,1.3,0.2]|
|              4.6|             3.1|              1.5|             0.2|     0|[4.6,3.1,1.5,0.2]|
|              5.0|             3.6|              1.4|             0.2|     0|[5.0,3.6,1.4,0.2]|
|              5.4|             3.9|              1.7|             0.4|     0|[5.4,3.9,1.7,0.4]|
|              4.6|             3.4|              1.4|             0.3|     0|[4.6,3.4,1.4,0.3]|
|              5.0|           

In [12]:
modeldata = output.select('Features','target')

In [13]:
modeldata.show()

+-----------------+------+
|         Features|target|
+-----------------+------+
|[5.1,3.5,1.4,0.2]|     0|
|[4.9,3.0,1.4,0.2]|     0|
|[4.7,3.2,1.3,0.2]|     0|
|[4.6,3.1,1.5,0.2]|     0|
|[5.0,3.6,1.4,0.2]|     0|
|[5.4,3.9,1.7,0.4]|     0|
|[4.6,3.4,1.4,0.3]|     0|
|[5.0,3.4,1.5,0.2]|     0|
|[4.4,2.9,1.4,0.2]|     0|
|[4.9,3.1,1.5,0.1]|     0|
|[5.4,3.7,1.5,0.2]|     0|
|[4.8,3.4,1.6,0.2]|     0|
|[4.8,3.0,1.4,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[5.8,4.0,1.2,0.2]|     0|
|[5.7,4.4,1.5,0.4]|     0|
|[5.4,3.9,1.3,0.4]|     0|
|[5.1,3.5,1.4,0.3]|     0|
|[5.7,3.8,1.7,0.3]|     0|
|[5.1,3.8,1.5,0.3]|     0|
+-----------------+------+
only showing top 20 rows



In [14]:
# split data
train_data, test_data = modeldata.randomSplit([0.8,0.2])

In [15]:
train_data.show()

+-----------------+------+
|         Features|target|
+-----------------+------+
|[4.3,3.0,1.1,0.1]|     0|
|[4.4,2.9,1.4,0.2]|     0|
|[4.4,3.0,1.3,0.2]|     0|
|[4.4,3.2,1.3,0.2]|     0|
|[4.5,2.3,1.3,0.3]|     0|
|[4.6,3.1,1.5,0.2]|     0|
|[4.6,3.4,1.4,0.3]|     0|
|[4.6,3.6,1.0,0.2]|     0|
|[4.7,3.2,1.3,0.2]|     0|
|[4.7,3.2,1.6,0.2]|     0|
|[4.8,3.0,1.4,0.1]|     0|
|[4.8,3.0,1.4,0.3]|     0|
|[4.8,3.1,1.6,0.2]|     0|
|[4.8,3.4,1.9,0.2]|     0|
|[4.9,3.0,1.4,0.2]|     0|
|[4.9,3.6,1.4,0.1]|     0|
|[5.0,3.2,1.2,0.2]|     0|
|[5.0,3.4,1.5,0.2]|     0|
|[5.0,3.4,1.6,0.4]|     0|
|[5.0,3.5,1.6,0.6]|     0|
+-----------------+------+
only showing top 20 rows



In [16]:
# naive bayes model
from pyspark.ml.classification import NaiveBayes

In [17]:
nb = NaiveBayes(featuresCol='Features', labelCol='target')

In [18]:
nb = nb.fit(train_data)

In [19]:
# prediction
y_pred = nb.transform(test_data)

In [20]:
y_pred.show()

+-----------------+------+--------------------+--------------------+----------+
|         Features|target|       rawPrediction|         probability|prediction|
+-----------------+------+--------------------+--------------------+----------+
|[4.6,3.2,1.4,0.2]|     0|[-11.400041543354...|[0.66076488616641...|       0.0|
|[4.8,3.4,1.6,0.2]|     0|[-12.147731909479...|[0.66227351459691...|       0.0|
|[4.9,2.4,3.3,1.0]|     1|[-17.315904889134...|[0.09245640211949...|       1.0|
|[4.9,3.1,1.5,0.1]|     0|[-11.338548782834...|[0.67354232107440...|       0.0|
|[4.9,3.1,1.5,0.2]|     0|[-11.698604295395...|[0.64370607542571...|       0.0|
|[5.0,2.0,3.5,1.0]|     1|[-17.341020406410...|[0.06534669275611...|       1.0|
|[5.0,3.0,1.6,0.2]|     0|[-11.855244888641...|[0.61674588291459...|       0.0|
|[5.0,3.3,1.4,0.2]|     0|[-11.792488155854...|[0.69148449028946...|       0.0|
|[5.0,3.5,1.3,0.3]|     0|[-12.175466449475...|[0.70467887273912...|       0.0|
|[5.1,3.4,1.5,0.2]|     0|[-12.166333338

In [21]:
# confusion matrix
y_pred.groupBy('target', 'prediction').count().show()

+------+----------+-----+
|target|prediction|count|
+------+----------+-----+
|     1|       2.0|    2|
|     0|       0.0|   15|
|     1|       1.0|    7|
|     2|       2.0|    7|
+------+----------+-----+



In [22]:
from sklearn.metrics import confusion_matrix
pred = y_pred.select("prediction").collect()
orig = y_pred.select("target").collect()
print(confusion_matrix(orig, pred))

[[15  0  0]
 [ 0  7  2]
 [ 0  0  7]]


In [23]:
# evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [24]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', predictionCol='prediction'
)

In [29]:
evaluator.metricName

Param(parent='MulticlassClassificationEvaluator_4d276912f91b', name='metricName', doc='metric name in evaluation (f1|accuracy|weightedPrecision|weightedRecall|weightedTruePositiveRate| weightedFalsePositiveRate|weightedFMeasure|truePositiveRateByLabel| falsePositiveRateByLabel|precisionByLabel|recallByLabel|fMeasureByLabel| logLoss|hammingLoss)')

In [25]:
accuracy = evaluator.evaluate(y_pred)
precision = evaluator.evaluate(y_pred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(y_pred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(y_pred, {evaluator.metricName: 'f1'})

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Accuracy: 0.935483870967742
Precision: 0.949820788530466
Recall: 0.935483870967742
F1 Score: 0.935483870967742


In [26]:
# close connection to spark
spark.stop()

In [27]:
fin = dt.datetime.now()

print(str(fin - ini))

0:00:22.009637
